# Structural Lib Colab Demo (Visual Reports)

This notebook installs the package, runs a small job, and renders the HTML report.


In [ ]:
!pip -q install "structural-lib-is456[dxf]"


: 

In [ ]:
from structural_lib import api
print('Version:', api.get_library_version())


In [ ]:
from structural_lib import flexure

res = flexure.design_singly_reinforced(
    b=300, d=450, d_total=500, mu_knm=150, fck=25, fy=500
)
print('Ast required (mm^2):', round(res.ast_required))
print('Status:', 'OK' if res.is_safe else res.error_message)


In [ ]:
import json

job = {
    'schema_version': 1,
    'code': 'IS456',
    'units': 'IS456',
    'job_id': 'colab_job_001',
    'beam': {
        'b_mm': 230,
        'D_mm': 500,
        'd_mm': 450,
        'd_dash_mm': 50,
        'fck_nmm2': 25,
        'fy_nmm2': 500,
        'asv_mm2': 100,
    },
    'cases': [
        {'case_id': 'LC1', 'mu_knm': 80, 'vu_kn': 55},
        {'case_id': 'LC2', 'mu_knm': 110, 'vu_kn': 75},
        {'case_id': 'LC3', 'mu_knm': 140, 'vu_kn': 95},
    ],
}

with open('job.json', 'w', encoding='utf-8') as f:
    json.dump(job, f, indent=2)

print('Wrote job.json')


In [ ]:
!python -m structural_lib job job.json -o ./job_out
!python -m structural_lib critical ./job_out --top 5 --format=csv -o critical.csv
!python -m structural_lib report ./job_out --format=html -o report.html


In [ ]:
from IPython.display import HTML, IFrame, display
from pathlib import Path
import pandas as pd

display(HTML(Path('report.html').read_text(encoding='utf-8')))
pd.read_csv('critical.csv').head(5)


In [ ]:
# If inline HTML does not render, use an iframe:
IFrame('report.html', width=900, height=600)


## BBS + DXF + Mark Consistency (Optional)

This section generates a small CSV, runs design → BBS → DXF, then checks
that bar marks match between the CSV and DXF callouts.


In [ ]:
import csv

rows = [{
    'BeamID': 'B1',
    'Story': 'G',
    'b': 230,
    'D': 500,
    'Span': 4000,
    'Cover': 40,
    'fck': 25,
    'fy': 500,
    'Mu': 120,
    'Vu': 80,
    'Ast_req': 0,
    'Asc_req': 0,
    'Stirrup_Dia': 8,
    'Stirrup_Spacing': 150,
}]

with open('beams_small.csv', 'w', newline='') as f:
    w = csv.DictWriter(f, fieldnames=rows[0].keys())
    w.writeheader()
    w.writerows(rows)

print('Wrote beams_small.csv')


In [ ]:
!python -m structural_lib design beams_small.csv -o results_small.json
!python -m structural_lib bbs results_small.json -o schedule_small.csv
!python -m structural_lib dxf results_small.json -o drawings_small.dxf
!python -m structural_lib mark-diff --bbs schedule_small.csv --dxf drawings_small.dxf --format json -o mark_diff.json


In [ ]:
import json
import pandas as pd
from IPython.display import display

with open('mark_diff.json', 'r', encoding='utf-8') as f:
    mark_diff = json.load(f)

print(mark_diff)
display(pd.read_csv('schedule_small.csv').head(5))


In [ ]:
from pathlib import Path

try:
    import ezdxf
    from ezdxf.addons import drawing
    from ezdxf.addons.drawing.matplotlib import MatplotlibBackend
    import matplotlib

    matplotlib.use('Agg')
    import matplotlib.pyplot as plt

    doc = ezdxf.readfile('drawings_small.dxf')
    msp = doc.modelspace()

    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_aspect('equal')
    ax.axis('off')

    ctx = drawing.RenderContext(doc)
    backend = MatplotlibBackend(ax)
    drawing.Frontend(ctx, backend).draw_layout(msp)
    backend.finalize()
    ax.autoscale()

    out_path = Path('drawings_small.png')
    fig.savefig(out_path, dpi=200, bbox_inches='tight', pad_inches=0.1)
    plt.close(fig)
    print(f'Rendered {out_path}')
except Exception as exc:
    print('Render skipped:', exc)


In [ ]:
from IPython.display import Image, display
from pathlib import Path

if Path('drawings_small.png').exists():
    display(Image('drawings_small.png'))
